# Alignement StatsBomb ↔ SkillCorner via `matching=statsbomb`  

**Objectif / 目标**：Réunir les données StatsBomb et SkillCorner via **correspondance d’ID** ou **correspondance floue**.  
从 StatsBomb 和 SkillCorner 爬取数据后，通过 **ID 匹配** 或 **模糊匹配** 将两方数据对应起来。

**Découverte / 发现**：SkillCorner propose `get_teams(params={'matching': 'statsbomb'})` : équipes liées à StatsBomb, certaines avec `statsbomb_id` pour une correspondance précise.  
SkillCorner 提供该 API，返回与 StatsBomb 有关联的球队，部分带 `statsbomb_id` 字段，可直接用于 ID 精确匹配。

In [1]:
import sys
import os

# Racine projet (depuis API ou racine) / 项目根路径
_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "API" else os.getcwd()
sys.path.insert(0, _root)

from skillcorner.client import SkillcornerClient
from config import SKILLCORNER_USERNAME, SKILLCORNER_PASSWORD

client = SkillcornerClient(username=SKILLCORNER_USERNAME, password=SKILLCORNER_PASSWORD)
print("SkillCorner client connecté / SkillCorner 客户端已连接")

SkillCorner client connecté / SkillCorner 客户端已连接


/Users/hazelchan00324/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1. Nouvelle méthode / 新方法：`matching=statsbomb` + filtre Ligue 1 2025-2026

Récupérer toutes les équipes `matching=statsbomb`, puis **filtrer sur Ligue 1 2025-2026** (même périmètre que l’ancienne méthode).  
先获取**全量** matching=statsbomb 球队，再**筛选到 Ligue 1 2025-2026 赛季**，与原方法范围一致。

- **Complet / 全量**：~4950 équipes liées à StatsBomb  
- **Après filtre / 筛选后**：uniquement les équipes Ligue 1 de cette saison（同 `competition_edition`）

In [2]:
# 1) Complet matching=statsbomb / 全量
teams_sb_all = client.get_teams(params={'matching': 'statsbomb'})
sb_lookup = {t['id']: t for t in teams_sb_all}  # sc_id -> infos complètes / 完整信息

# 2) Saison Ligue 1 2025-2026 (même périmètre) / 获取赛季（与原方法相同）
from src.skillcorner_ingestion import find_ligue1_edition
edition_id, _ = find_ligue1_edition(client)

# 3) Liste équipes de la saison / 该赛季球队列表
teams_ligue1 = client.get_teams(params={'competition_edition': edition_id}) if edition_id else []

# 4) Filtre + enrichissement statsbomb_id / 筛选并补充 statsbomb_id
teams = []
for t in teams_ligue1:
    enriched = sb_lookup.get(t['id'], t)
    teams.append({**t, 'statsbomb_id': enriched.get('statsbomb_id')})

print(f"【Nouvelle méthode / 新方法】matching=statsbomb 全量: {len(teams_sb_all)} équipes / 支")
print(f"【Nouvelle méthode / 新方法】après filtre Ligue 1 2025-2026 / 筛选后: {len(teams)} équipes / 支")

# Stats avec/sans statsbomb_id / 统计有/无 statsbomb_id
with_sb_id = [t for t in teams if t.get('statsbomb_id')]
without_sb_id = [t for t in teams if not t.get('statsbomb_id')]
print(f"\nAvec statsbomb_id (correspondance ID directe) / 有 statsbomb_id（可直接 ID 匹配）: {len(with_sb_id)}")
print(f"Sans statsbomb_id (correspondance floue) / 无 statsbomb_id（需模糊匹配）: {len(without_sb_id)}")

📡 Searching for Ligue 1 competition edition...
 Fetching SkillCorner seasons...


   Found 33 seasons
   Checking season: 2026 (id=130)
 Fetching SkillCorner competitions...


   Found 42 competitions
   Checking season: 2025/2026 (id=129)
 Fetching SkillCorner competitions...


   Found 134 competitions
   Found competition: Ligue 1 (id=3)


   Found edition: FRA - Ligue 1 - 2025/2026 (id=1232)


【Nouvelle méthode / 新方法】matching=statsbomb 全量: 4952 équipes / 支
【Nouvelle méthode / 新方法】après filtre Ligue 1 2025-2026 / 筛选后: 18 équipes / 支

Avec statsbomb_id (correspondance ID directe) / 有 statsbomb_id（可直接 ID 匹配）: 18
Sans statsbomb_id (correspondance floue) / 无 statsbomb_id（需模糊匹配）: 0


In [3]:
# Exemples avec/sans statsbomb_id / 有/无 statsbomb_id 的示例
print("Exemples avec statsbomb_id / 有 statsbomb_id 的示例:")
for t in with_sb_id[:5]:
    print(f"  SC id={t['id']}, name={t['name']}, statsbomb_id={t['statsbomb_id']}")

print("\nExemples sans statsbomb_id / 无 statsbomb_id 的示例:")
for t in without_sb_id[:5]:
    print(f"  SC id={t['id']}, name={t['name']}, statsbomb_id={t.get('statsbomb_id')}")

Exemples avec statsbomb_id / 有 statsbomb_id 的示例:
  SC id=85, name=RC Lens, statsbomb_id=168
  SC id=65, name=AJ Auxerre, statsbomb_id=167
  SC id=66, name=FC Nantes, statsbomb_id=144
  SC id=69, name=Le Havre AC, statsbomb_id=164
  SC id=70, name=OGC Nice, statsbomb_id=136

Exemples sans statsbomb_id / 无 statsbomb_id 的示例:


## 2. Construction de la correspondance équipes / 构建 SkillCorner ↔ StatsBomb 球队映射

Pour les équipes avec `statsbomb_id` : mapping direct `(skillcorner_team_id, statsbomb_team_id)` pour l’ingestion, en remplacement du matching par nom.  
对有 `statsbomb_id` 的球队，可直接建立映射，用于后续 ingest 时的精确匹配，替代按名称的模糊匹配。

In [4]:
# Mapping SC -> SB / 映射: skillcorner_team_id -> statsbomb_team_id
sc_to_sb_team = {t['id']: int(t['statsbomb_id']) for t in teams if t.get('statsbomb_id')}

# Mapping inverse SB -> SC / 反向: statsbomb_team_id -> skillcorner_team_id
sb_to_sc_team = {int(t['statsbomb_id']): t['id'] for t in teams if t.get('statsbomb_id')}

print(f"Paires équipes à correspondance ID directe / 可直接 ID 匹配的球队对数: {len(sc_to_sb_team)}")
print("\nExemples de mapping (SC -> SB) / 映射示例 (SC -> SB):")
for i, (sc_id, sb_id) in enumerate(list(sc_to_sb_team.items())[:5]):
    name = next(t['name'] for t in teams if t['id'] == sc_id)
    print(f"  {name}: SC {sc_id} -> SB {sb_id}")

Paires équipes à correspondance ID directe / 可直接 ID 匹配的球队对数: 18

Exemples de mapping (SC -> SB) / 映射示例 (SC -> SB):
  RC Lens: SC 85 -> SB 168
  AJ Auxerre: SC 65 -> SB 167
  FC Nantes: SC 66 -> SB 144
  Le Havre AC: SC 69 -> SB 164
  OGC Nice: SC 70 -> SB 136


## 3. Différences avec l’ingestion actuelle / 与当前 ingest 方式的区别

| Dimension | Actuel `skillcorner_ingestion.py` | Nouveau `matching=statsbomb` |
|-----------|-----------------------------------|-----------------------------|
| **Source / 数据源** | `competition_edition` → une seule ligue/saison | `matching=statsbomb` → toutes équipes liées StatsBomb |
| **Stratégie / 匹配策略** | Matching par nom LIKE (flou) | Si `statsbomb_id` → ID exact, sinon nom flou |
| **Usage / 用途** | Ingest Ligue 1 ciblée | Mapping SC↔SB global, multi-ligues |

In [5]:
# Équipes Ligue 1 2025-2026 avec statsbomb_id (après filtre) / 筛选后新方法
print("Équipes Ligue 1 2025-2026 avec statsbomb_id / Ligue 1 2025-2026 赛季中有 statsbomb_id 的球队:")
for t in with_sb_id:
    print(f"  {t['name']}: statsbomb_id={t['statsbomb_id']}")

Équipes Ligue 1 2025-2026 avec statsbomb_id / Ligue 1 2025-2026 赛季中有 statsbomb_id 的球队:
  RC Lens: statsbomb_id=168
  AJ Auxerre: statsbomb_id=167
  FC Nantes: statsbomb_id=144
  Le Havre AC: statsbomb_id=164
  OGC Nice: statsbomb_id=136
  FC Lorient: statsbomb_id=165
  Toulouse FC: statsbomb_id=130
  LOSC Lille: statsbomb_id=143
  FC Metz: statsbomb_id=141
  Angers SCO: statsbomb_id=145
  AS Monaco: statsbomb_id=129
  Olympique de Marseille: statsbomb_id=147
  Stade Rennais FC: statsbomb_id=134
  Stade Brestois 29: statsbomb_id=153
  Paris Saint-Germain: statsbomb_id=131
  RC Strasbourg Alsace: statsbomb_id=138
  Paris FC: statsbomb_id=151
  Olympique Lyonnais: statsbomb_id=137


## 4. Requête BDD : résultat méthode actuelle / 查询数据库：原方法结果

La méthode actuelle utilise `get_teams(competition_edition=X)` puis associe aux équipes StatsBomb par **matching flou sur le nom (LIKE)**.  
原方法使用 `competition_edition` 获取球队，再按**球队名称 LIKE 模糊匹配**与 StatsBomb 关联。

In [6]:
# 1) Méthode actuelle : competition_edition Ligue 1 2025-2026 / 原方法：获取 Ligue 1 球队
teams_original = teams_ligue1
print(f"【Méthode actuelle / 原方法】get_teams(competition_edition={edition_id}) → {len(teams_original)} équipes / 支")

# 2) État actuel des équipes en BDD / 查询数据库中球队状态
from src.database import get_connection, table

conn = get_connection()
cur = conn.cursor()
# Toutes équipes (pour comparaison) ; avec statsbomb_team_id pour matching ID / 查所有球队（用于对比）；有 SB ID 用于精确匹配
cur.execute(f"""
    SELECT team_id, team_name, statsbomb_team_id, skillcorner_team_id
    FROM {table('teams')}
    ORDER BY team_name
""")
db_teams_all = cur.fetchall()
db_teams = [(r[0], r[1], r[2], r[3]) for r in db_teams_all if r[2] is not None]  # Avec SB ID / 有 StatsBomb ID
conn.close()

print(f"\n【BDD / 数据库】Total équipes: {len(db_teams_all)} / 球队总数: {len(db_teams_all)} 支")
print(f"【BDD】Avec StatsBomb ID: {len(db_teams)} / 有 StatsBomb ID: {len(db_teams)} 支")
print(f"【BDD】Avec SkillCorner ID: {sum(1 for r in db_teams_all if r[3])} / 有 SkillCorner ID: {sum(1 for r in db_teams_all if r[3])} 支")

【Méthode actuelle / 原方法】get_teams(competition_edition=1232) → 18 équipes / 支

【BDD / 数据库】Total équipes: 18 / 球队总数: 18 支
【BDD】Avec StatsBomb ID: 0 / 有 StatsBomb ID: 0 支
【BDD】Avec SkillCorner ID: 18 / 有 SkillCorner ID: 18 支


## 5. Comparaison ancienne vs nouvelle méthode / 新旧方法对比

Pour les équipes en BDD / 针对数据库中的球队：
- **Méthode actuelle / 原方法** : `competition_edition` → matching SC par nom (LIKE flou)
- **Nouvelle méthode / 新方法** : `matching=statsbomb` → matching par `statsbomb_id` (ID précis)

In [7]:
# Nouvelle méthode : SB -> SC / 新方法 statsbomb_team_id -> skillcorner_team_id
sb_to_sc = sb_to_sc_team  # Construit ci-dessus / 已在上面构建

# Lookup par nom pour fallback / 建立 SC 球队名映射，方便按名称查找
sc_teams_by_name = {t.get("name", "").lower(): t for t in teams}

# Pour chaque équipe en BDD, tester les deux méthodes / 对每支球队检查两种方法能否匹配
# Si sb_id existe : db_teams ; sinon db_teams_all + matching par nom / 有 sb_id 用 db_teams，否则按名称对比
comparison = []
for team_id, team_name, sb_id, sc_id_db in db_teams:
    sb_id = int(sb_id) if sb_id else None
    if not sb_id:
        continue
    # Nouvelle méthode : statsbomb_id -> sc_id direct ? / 新方法能否通过 statsbomb_id 找到 SC id
    sc_id_new = sb_to_sc.get(sb_id)
    match_new = "ID précis / ID 精确匹配" if sc_id_new else "Non / 无"
    match_original = "Nom flou / 名称模糊匹配" if sc_id_db else "Non matché / 未匹配"
    comparison.append({
        "team_name": team_name,
        "sb_id": sb_id,
        "sc_id_db": sc_id_db,
        "sc_id_new": sc_id_new,
        "原方法": match_original,
        "新方法": match_new,
        "新方法更优": sc_id_new and not sc_id_db,
        "一致": (sc_id_db == sc_id_new) if (sc_id_db and sc_id_new) else False,
    })

# Fallback : pas de statsbomb_team_id en BDD -> matching par nom / 若无 statsbomb_team_id 则按名称查找
# Alias connus : abréviation DB <-> racine SC (ex. Rennes/Rennais) / 已知别名 DB 简称 <-> SC 词根
_NAME_ALIASES = [("rennes", "rennais")]

def _name_matches(db_name, sc_name):
    """Matching strict par nom, éviter faux positifs (ex. Lens vs Alajuelense) / 严格名称匹配，避免误匹配"""
    a, b = (db_name or "").lower(), (sc_name or "").lower()
    if a == b:
        return True
    # Alias : Rennes <-> Stade Rennais FC / 已知别名
    for short, long_root in _NAME_ALIASES:
        if (a == short and long_root in b) or (b == short and long_root in a):
            return True
    # Sous-chaîne = mot entier ou début / 子串匹配时必须是独立词或开头
    if a in b:
        return b.startswith(a) or f" {a}" in b or b.endswith(f" {a}")
    if b in a:
        return a.startswith(b) or f" {b}" in a or a.endswith(f" {b}")
    return False

if not comparison and db_teams_all:
    print("(Pas de statsbomb_team_id en BDD, fallback par nom / 数据库中无 statsbomb_team_id，按名称在 matching=statsbomb 中查找)\n")
    for team_id, team_name, sb_id, sc_id_db in db_teams_all:
        name_lower = (team_name or "").lower()
        sc_match = None
        for sc_t in teams:
            if _name_matches(team_name, sc_t.get("name")):
                sc_match = sc_t
                break
        if sc_match:
            sb_id_found = sc_match.get("statsbomb_id")
            sb_id_found = int(sb_id_found) if sb_id_found else None
            comparison.append({
                "team_name": team_name,
                "sb_id": sb_id_found,
                "sc_id_db": sc_id_db,
                "sc_id_new": sc_match.get("id") if sb_id_found else None,
                "原方法": "Nom flou / 名称模糊匹配" if sc_id_db else "Non matché / 未匹配",
                "新方法": "ID précis / ID 精确匹配" if sb_id_found else "Par nom / 需名称匹配",
                "新方法更优": sb_id_found and not sc_id_db,
                "一致": (sc_id_db == sc_match.get("id")) if (sc_id_db and sb_id_found) else False,
            })

# Statistiques / 统计
n_total = len(comparison)
if n_total == 0:
    print("Impossible de comparer / 无法进行对比：BDD vide ou noms non matchables.")
else:
    n_original_matched = sum(1 for c in comparison if c["sc_id_db"])
    n_new_matched = sum(1 for c in comparison if c["sc_id_new"])
    n_both = sum(1 for c in comparison if c["sc_id_db"] and c["sc_id_new"])
    n_new_only = sum(1 for c in comparison if c["sc_id_new"] and not c["sc_id_db"])
    n_mismatch = sum(1 for c in comparison if c["sc_id_db"] and c["sc_id_new"] and c["sc_id_db"] != c["sc_id_new"])

    print("=" * 60)
    print("📊 Comparaison ancienne vs nouvelle / 新旧方法对比结果")
    print("=" * 60)
    print(f"  Équipes comparées / 参与对比的球队总数.............. {n_total}")
    print(f"  Méthode actuelle matchée (skillcorner_team_id)..... {n_original_matched} ({100*n_original_matched/n_total:.1f}%)")
    print(f"  Nouvelle méthode (statsbomb_id) / 新方法可 ID 匹配.. {n_new_matched} ({100*n_new_matched/n_total:.1f}%)")
    print(f"  Les deux cohérents / 两种方法都匹配且一致.......... {n_both}")
    print(f"  Seule nouvelle matchée / 新方法能匹配原方法未匹配.. {n_new_only}")
    print(f"  ID SC divergents / 匹配结果不一致 (SC id 不同).... {n_mismatch}")
    print()

(Pas de statsbomb_team_id en BDD, fallback par nom / 数据库中无 statsbomb_team_id，按名称在 matching=statsbomb 中查找)

📊 Comparaison ancienne vs nouvelle / 新旧方法对比结果
  Équipes comparées / 参与对比的球队总数.............. 18
  Méthode actuelle matchée (skillcorner_team_id)..... 18 (100.0%)
  Nouvelle méthode (statsbomb_id) / 新方法可 ID 匹配.. 18 (100.0%)
  Les deux cohérents / 两种方法都匹配且一致.......... 18
  Seule nouvelle matchée / 新方法能匹配原方法未匹配.. 0
  ID SC divergents / 匹配结果不一致 (SC id 不同).... 18



In [8]:
import pandas as pd

if comparison:
    df = pd.DataFrame(comparison)
    # Colonnes bilingues fr/zh / 双语列名
    df_display = df[["team_name", "sb_id", "sc_id_db", "sc_id_new", "原方法", "新方法"]].head(20)
    df_display.columns = ["Équipe/球队", "SB_id", "SC_id(BDD)", "SC_id(nouveau)", "Méthode actuelle/原方法", "Nouvelle/新方法"]
    display(df_display)

    if n_new_only > 0:
        print("\n⚠️ Nouvelle méthode matchée, ancienne non (à corriger) / 新方法能匹配但原方法未匹配:")
        for c in [x for x in comparison if x["新方法更优"]]:
            print(f"   {c['team_name']}: sb_id={c['sb_id']} → sc_id={c['sc_id_new']} (ancien: vide / 原: 无)")
else:
    print("Aucune donnée à afficher. Vérifier StatsBomb ingest. / 无对比数据，请检查 StatsBomb ingest。")

,Équipe/球队,SB_id,SC_id(BDD),SC_id(nouveau),Méthode actuelle/原方法,Nouvelle/新方法
0,AS Monaco,129,79,75,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配
1,Angers,145,80,98,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配
2,Auxerre,167,327,65,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配
3,Le Havre,164,78,69,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配
4,Lens,168,98,85,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配
5,Lille,143,74,78,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配
6,Lorient,165,76,72,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配
7,Lyon,137,89,73,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配
8,Marseille,147,97,80,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配
9,Metz,141,73,97,Nom flou / 名称模糊匹配,ID précis / ID 精确匹配


## 6. Résumé : Comparaison des deux méthodes (Ligue 1 2025-2026)  
## 总结：两种方法对比（均限定 Ligue 1 2025-2026）

| Dimension / 维度 | Méthode actuelle / 原方法 | Nouvelle méthode / 新方法 |
|-----------------|--------------------------|--------------------------|
| **Périmètre / 数据范围** | Ligue 1 2025-2026 (`competition_edition`) | Idem, filtre après `matching=statsbomb` |
| **API** | `get_teams(competition_edition=X)` | `get_teams(matching='statsbomb')` + filtre |
| **Correspondance / 匹配依据** | Nom LIKE (flou) / 名称模糊匹配 | `statsbomb_id` (précis) / 精确匹配 |
| **Précision / 精确度** | Risque d’erreur / 可能误匹配 | ID 1:1 / 一一对应 |
| **Recommandation / 建议** | Garder pour liste équipes / 保留获取球队 | **Prioriser statsbomb_id** / 优先 ID 匹配 |